In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

# A Few Shot Learning

In [2]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

😍,👏,✨


# 프롬프트 엔지니어링 모범 사례

## 명확한 지침 작성

예시:

-----------------------
프롬프트:

피보나치 수열을 계산하는 코드를 작성하세요.

더 나은 프롬프트:

피보나치 수열을 효율적으로 계산하는 TypeScript 함수를 작성하세요. 각 부분이 무엇을 하고 왜 그렇게 작성되었는지 설명하는 주석을 충분히 추가하세요.

----------------------

프롬프트:

회의록을 요약하세요.

더 나은 프롬프트:

회의록을 한 문단으로 요약하세요. 그런 다음 발표자와 각 발표자의 주요 요점을 나열한 마크다운 목록을 작성하세요. 마지막으로 발표자가 제안한 다음 단계나 실행 항목이 있다면 이를 나열하세요.


## 역할 놀이

예시:

-----------------------

시스템 메시지: 내가 무언가를 작성하는 데 도움을 요청하면, 당신은 모든 문단에 적어도 하나의 농담이나 장난스러운 댓글이 포함된 문서를 작성하여 응답할 것입니다.

----------------------


## 입력 텍스트 분할

예시:

------------------------

사용자 메시지: 세 개의 큰따옴표로 구분된 텍스트를 하이쿠로 요약하세요.

"""여기에 텍스트를 삽입하세요"""

------------------------

시스템 메시지: 동일한 주제에 대한 두 개의 기사(XML 태그로 구분됨)가 제공됩니다. 먼저 각 기사의 주장을 요약하세요. 그런 다음 어떤 기사가 더 나은 주장을 펼치는지와 그 이유를 설명하세요.

사용자 메시지:

\<article> 첫 번째 기사를 여기에 삽입하세요 \</article>

\<article> 두 번째 기사를 여기에 삽입하세요 \</article>

------------------------

## 작업을 완료하기 위한 단계와 프로세스 설명

예시:

--------------------------------

시스템 메시지:
다음 단계별 지침을 사용하여 사용자 입력에 응답하세요.

1단계 - 사용자가 세 개의 큰따옴표로 구분된 텍스트를 제공합니다. 이 텍스트를 "요약:"이라는 접두어와 함께 한 문장으로 요약하세요.

2단계 - 1단계의 요약을 스페인어로 번역하고 "번역:"이라는 접두어를 추가하세요.

---------------------------------

## 몇 샷 학습 사용

